## Homework 1: Decoding Strategies and Emoji Classification

This homework combines decoding strategies (greedy and temperature sampling) and emoji classification using embeddings.

##### Submission Instructions
1. Complete all the functions marked with "Your code here".
2. Run all cells to make sure your outputs are correct.
3. Answer all output questions in markdown.
4. Due on 13th Feb. 2026, Friday at 23:59pm
5. Submit your solution notebook to "Canvas -> HW1-Assignment"
6. **Prepend your NUS userID to the filename, i.e., "`a0123456b_HW.ipynb`"**. Only ipython notebooks are accepted.


## Part 1: Decoding Strategies (5 pts)
In Lecture 3, decoding happens after a final softmax layer, and different strategies can be used.
You will implement greedy decoding and temperature sampling, then interpret the outputs.


Lecture 3 decoding strategies:
- Greedy search: choose the highest-probability token.
- Beam search: keep the top-N partial sequences.
- Temperature sampling: re-scale logits before sampling.


In [1]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Example logits (pre-softmax scores) for 10 tokens
tokens = ["token_" + str(i) for i in range(10)]
logits = np.array([5.0, 4.0, 3.0, 2.0, 1.0, 0.5, 0.3, 0.2, 0.1, 0.05], dtype=np.float32)


def softmax(x):
    """Compute a numerically stable softmax for a 1D array."""
    shifted = x - np.max(x)
    exp = np.exp(shifted)
    return exp / np.sum(exp)


### Task 1.1: Implement temperature scaling
Complete the function below to apply temperature scaling and return probabilities.

In [3]:
def apply_temperature(logits, temperature):
    """Apply temperature scaling to logits and return probabilities."""
    # Your code here
    scaled_logits = logits / temperature
    
    # Apply softmax to get probabilities
    probs = softmax(scaled_logits)
    
    return probs
    pass


In [4]:
temperatures = [0.1, 0.5, 1.0, 2.0]

for temp in temperatures:
    probs = apply_temperature(logits, temp)
    top_idx = int(np.argmax(probs))
    print(f"Temperature {temp} argmax token: {tokens[top_idx]} | prob={probs[top_idx]:.4f}")
    print(f"Sum of probabilities: {np.sum(probs):.4f}")
    print("-")


Temperature 0.1 argmax token: token_0 | prob=1.0000
Sum of probabilities: 1.0000
-
Temperature 0.5 argmax token: token_0 | prob=0.8644
Sum of probabilities: 1.0000
-
Temperature 1.0 argmax token: token_0 | prob=0.6195
Sum of probabilities: 1.0000
-
Temperature 2.0 argmax token: token_0 | prob=0.3578
Sum of probabilities: 1.0000
-


### Task 1.2: Implement greedy decoding
Complete the function below to choose the highest-probability token (greedy decoding).

In [5]:
def greedy_decode(probabilities, tokens):
    """Return the single highest-probability token and its index."""
    # Your code here
    # Find the index of the highest probability
    top_index = int(np.argmax(probabilities))
    
    # Get the corresponding token
    top_token = tokens[top_index]
    return top_token, top_index
    pass


In [6]:
for temp in temperatures:
    probs = apply_temperature(logits, temp)
    top_token, top_idx = greedy_decode(probs, tokens)
    print(f"Temperature {temp} greedy token: {top_token} | prob={probs[top_idx]:.4f}")


Temperature 0.1 greedy token: token_0 | prob=1.0000
Temperature 0.5 greedy token: token_0 | prob=0.8644
Temperature 1.0 greedy token: token_0 | prob=0.6195
Temperature 2.0 greedy token: token_0 | prob=0.3578


### Task 1.3: Implement sampling from probabilities
Complete the sampling function to draw tokens from the distribution.


In [9]:
def sample_tokens(probabilities, n_samples=10):
    """Sample tokens based on their probabilities."""
    # Your code here
    sampled_indices = np.random.choice(len(probabilities),
    size=n_samples,
    p=probabilities         
    )
    
    # Convert indices to token names
    sampled_tokens = np.array([tokens[idx] for idx in sampled_indices])
    
    return sampled_tokens
    pass


In [10]:
print("Sampling 10 tokens with different temperatures:")
for temp in temperatures:
    probs = apply_temperature(logits, temp)
    samples = sample_tokens(probs, n_samples=10)
    print(f"\nTemperature {temp}:")
    print(f"Samples: {samples}")
    print(f"Frequency of token_0: {np.sum(samples == 'token_0')} out of 10")


Sampling 10 tokens with different temperatures:

Temperature 0.1:
Samples: ['token_0' 'token_0' 'token_0' 'token_0' 'token_0' 'token_0' 'token_0'
 'token_0' 'token_0' 'token_0']
Frequency of token_0: 10 out of 10

Temperature 0.5:
Samples: ['token_0' 'token_1' 'token_0' 'token_0' 'token_0' 'token_0' 'token_0'
 'token_0' 'token_0' 'token_0']
Frequency of token_0: 9 out of 10

Temperature 1.0:
Samples: ['token_0' 'token_0' 'token_0' 'token_0' 'token_0' 'token_1' 'token_0'
 'token_0' 'token_0' 'token_0']
Frequency of token_0: 9 out of 10

Temperature 2.0:
Samples: ['token_2' 'token_0' 'token_0' 'token_8' 'token_8' 'token_4' 'token_0'
 'token_0' 'token_2' 'token_1']
Frequency of token_0: 4 out of 10


### Task 1.4: Output questions
Answer the following based on the outputs you observe:
1. For which temperature is the distribution most peaked? Why?
2. Which temperature shows the most diverse samples?
3. Does greedy decoding change with temperature here? Explain based on the outputs.


##### your answers here

1->temperature0.1 has the most peaked. when temp i, logits are divided by a small number to widen disparity . so softmaxed values puts all probabilities to the highest token Result: token0 sampled 10 times/10 100% probability token0-0 prob temp=0.1 1.0000.

2->Temperature 2.0 has the most variety of samples. Sample 6 different tokens, token0, token1, token2, token4, token_8. The distribution is flattened and tends to favor the winning of lower-ranked tokens.

3->No, greedy decoding does not change with temperature. Greedy decode always chooses the token with the highest probable value. The original highest logit value (5.0) for token_0 still makes it the most probable token at each temperature.

From the earlier greedy output, all temperatures selected token_0 as the greedy choice

Temperature only affects the confidence (probability value), not which token is most likely





## Part 2: Emoji Classification (5 pts)
Modern business communication often includes emojis, especially in social media and customer interactions.
Let's build a system that can suggest appropriate emojis for business communications.


In [16]:
import numpy as np
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# Common business-appropriate emojis with their meanings
BUSINESS_EMOJIS = {
    "📈": "growth, success, improvement",
    "📉": "decline, decrease, loss",
    "👍": "approval, agreement, good",
    "🤝": "agreement, partnership, collaboration",
    "📊": "statistics, analytics, data",
    "🚀": "launch, growth, acceleration",
    "💪": "strength, determination, success",
}


def suggest_business_emojis(text, top_k=2):
    """
    Suggest appropriate business emojis for a given text using embedding similarity.

    Args:
        text (str): Input business text
        top_k (int): Number of emoji suggestions to return

    Returns:
        list: List of tuples (emoji, confidence_score) sorted by confidence

    Example:
        >>> suggest_business_emojis("Our Q4 sales increased by 25%")
        [("📈", 0.92), ("🚀", 0.85)]
    """
    # Your code here:
    # 1. Create embeddings for input text
    text_embedding = sentence_model.encode(text)
    
    # 2. Create embeddings for emoji meanings
    emoji_list = list(BUSINESS_EMOJIS.keys())
    emoji_meanings = list(BUSINESS_EMOJIS.values())
    meaning_embeddings = sentence_model.encode(emoji_meanings)
    
    # 3. Calculate similarities (cosine similarity)
    text_norm = text_embedding / np.linalg.norm(text_embedding)
    meaning_norms = meaning_embeddings / np.linalg.norm(meaning_embeddings, axis=1, keepdims=True)
    
    similarities = np.dot(meaning_norms, text_norm)
    
    # 4. Return top_k emoji suggestions with confidence scores
    top_indices = np.argsort(similarities)[::-1][:top_k]
    suggestions = [(emoji_list[idx], float(similarities[idx])) for idx in top_indices]
    
    return suggestions
    
    pass



# Example business messages
business_messages = [
    "We exceeded our quarterly targets by 30%",
    "New partnership announcement coming soon",
    "Customer satisfaction ratings dropped this month",
    "Innovative solution launched for enterprise clients",
    "Team collaboration has improved significantly",
]


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1942.85it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [17]:
# Test your emoji suggestion function
for message in business_messages:
    print(f"\nMessage: {message}")
    print("Suggested emojis:", suggest_business_emojis(message))



Message: We exceeded our quarterly targets by 30%
Suggested emojis: [('📈', 0.2681995630264282), ('🚀', 0.2089613825082779)]

Message: New partnership announcement coming soon
Suggested emojis: [('🤝', 0.6311315894126892), ('👍', 0.4295920133590698)]

Message: Customer satisfaction ratings dropped this month
Suggested emojis: [('📉', 0.20369338989257812), ('📊', 0.1753871738910675)]

Message: Innovative solution launched for enterprise clients
Suggested emojis: [('📈', 0.34140026569366455), ('🤝', 0.3062551021575928)]

Message: Team collaboration has improved significantly
Suggested emojis: [('🤝', 0.5045756697654724), ('📈', 0.3936906158924103)]


#### Task 2.2: Analysis Questions
Answer the following questions in your report:
1. What are the limitations of this emoji suggestion system?
2. How could this system be improved for specific business contexts (e.g., customer service vs. internal communication)?
3. What are the potential risks of using this automated emoji suggestion system in professional business communications, and how would you mitigate them?


##### your answer here


1->Limited Emoji Dictionary, cannot handle tone adequacy, semantic matching mismatch, and Embedding similarity is not fine tuned for moderate business syntax.

2->Stage-aware: Different emojis for greeting vs. resolution Sentiment-averse escalation detection: Flag if emoji might worsen situation.
Response templates: Pre-approved emoji combinations.

3->Since we are using embedded models the context can be inaccurate at times leading to misinterpretations, as the system is not detecting the sentiment relationship. Over engineering/automation can lead to brand and Reputation risks as well to mitigate these use sentiment relationship layer maybe, market it as an assistant not a decision maker. Final decision makers and risk analysers should be humans.

#### End of HW
